In [1]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
import logging
import time

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

# 高相似多少
coum=200
threshold_list = np.linspace(0.5,0.7,210)
# 加载保存的数组
lin = np.load("lin.npy")
# lin = np.linspace(0,3,401)
# lin = np.linspace(0.01,1,100)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
cells_name = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx', header=0).columns[2:] # 保证cells的名字从第一行的第三个开始
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [2]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [3]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [4]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [5]:
def alpha_pcc(a, lin, linename, dict_columns_split, TTT, data2, NNN, coum_list):
    alpha = []
    result_pcc = []
    pcc_split = []
    pre_dict = []
    ori_dict = []
    coum = 0
    if a != 0:
        for i in coum_list[0:a]:
            coum += i
    t_coum = coum_list[a]
    for alp in lin:
    # alp = 1
    # if alp == 1:
        pre = []
        for elem in range(39):
            pp = []
            if TTT[coum:(coum+t_coum)] == []:
                print("该进程有问题")
            for j in TTT[coum:(coum+t_coum)]: 
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
            pp = np.array(pp)  # 将 pp 转换为 numpy 数组
            # print(TTT[coum:(coum+t_coum)])
            # print(len(pp))
            # print(len(dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
            pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
        pre = np.array(pre)  # 将 pre 转换为 numpy 数组
        pre_dict.append(pre)
        ori_dict.append(data2[linename])
        pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    alpha.append(lin[pcc_split.index(np.max(pcc_split))])
    pre = pre_dict[pcc_split.index(np.max(pcc_split))]
    ori = ori_dict[pcc_split.index(np.max(pcc_split))]
    pcc=result_pcc[0]
    if pcc > 1:
        print(pcc, lin)
    temp_NNN = NNN[coum:(coum+t_coum)]
    temp_TTT = TTT[coum:(coum+t_coum)]
    del result_pcc, pcc_split, pp, coum, t_coum
    return alpha, pcc, temp_NNN, temp_TTT, pre, ori

def alp_mse_rmse(alpha, a, linename, dict_columns_split, TTT, coum_list):
    r1 = []  # 初始化 r1 为空列表
    alp = alpha[0]
    coum = 0
    if a != 0:
        for i in coum_list[0:a]:
            coum += i
    t_coum = coum_list[a]
    for elem in range(39):
        pp = []
        for j in TTT[coum:(coum+t_coum)]: 
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
    r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
    mse = np.mean(np.square(r1))
    rmse = math.sqrt(mse)
    del r1, pp,
    return alp, mse, rmse

# 单进程 分函数
def calculate_parameters(linename, a, TTT, data2, dict_columns_split, lin, NNN, coum_list):
    # 求PCC
    alpha, pcc, temp_NNN, temp_TTT, pre, ori = alpha_pcc(a, lin, linename, dict_columns_split, TTT, data2, NNN, coum_list)
    # 求MSE
    alp, mse, rmse = alp_mse_rmse(alpha, a, linename, dict_columns_split, TTT, coum_list)
    Name_mse, Name_pcc, Name_alp, Name_rmse, Name_NNN, Name_TTT, Name_pre, Name_ori = mse, pcc, alp, rmse, temp_NNN, temp_TTT, pre, ori
    # print("name:{},alp={},mse={},pcc={},rmse={}".format(linename, alp, mse, pcc,rmse))
    del mse, pcc, alp, rmse, temp_NNN, temp_TTT, pre, ori
    return [linename, Name_alp, Name_mse, Name_pcc, Name_rmse, Name_NNN, Name_TTT, Name_pre, Name_ori]

# 多进程
@timeit
def run_parallel(columns, a, lin, NNN, coum_list):
    with multiprocessing.Pool() as pool:
        tp = []
        for idx, column in enumerate(columns):
            tp.append((column, a+idx, TTT, data2, dict_columns_split, lin, NNN, coum_list))
        results = pool.starmap(calculate_parameters, tp)
    return results

In [6]:
for threshold in threshold_list:
    # 分离x y数据构造字典
    dict_columns_split = {} # 高相似字典
    # 计算x数据
    x = data2.drop(columns=columns)  # 其他所有的列
    coum_list = []
    for i in columns:
        # b 要预测协同得分那一列
        y = data2.loc[:, i]
        # A 权重
        x_i = x.copy()
        x_i['y'] = y

        # 高相似的    取高相似的药物对名字max1和对应的值max5
        tp = []
        for j in columns:
            if i != j:
                tp.append(j)
        df = pd.DataFrame(dict_lines[i], index = tp)
        df.sort_values(by=0, inplace=True, ascending=True)
        coum = len([x for x in df.T.values[0] if x >= threshold])
        max_l = df.index[-coum:]
        coum_list.append(coum)
        max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

        # 存储 对应数据
        count = 0 # 第几行 int
        for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
            dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
            dict_columns_split[f'{i},{j}']['y_line']=y[count]
            dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
            count += 1

    TTT = [] # 值列表
    cct = 0
    NNN = []
    for i in Tanimoto.index: 
        y = list(Tanimoto.iloc[:,cct])
        y.pop(cct) 
        tp_columns = list(columns.copy())
        tp_columns.pop(cct)
        df = pd.DataFrame(y, index=tp_columns)
        df.sort_values(by=0 , inplace=True, ascending=True) 
        max_l = df.index[-coum_list[cct]:]
        NNN.extend(max_l)
        # 计算max_5y
        max_5y = []
        for j in max_l:
            max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
        # 将max_5y加入TTT
        TTT += max_5y
        # 修改cct变量的值
        cct += 1
    
    try:
        a = 0
        results = run_parallel(columns, a, lin, NNN, coum_list)   
    except:
        print(threshold)
        print(coum_list)
        break

    rebuild_results = []
    for i in results:
        rebuild_results.append([i[0],i[1],i[2],i[3],i[4]])

    # 将结果保存到 CSV 文件中
    pd.DataFrame(rebuild_results).to_csv(f'./threshold/{threshold}_pcc_cpu_test_multi_threshold.csv', header=None, index=False)
    rebuild_list = []
    pccs = []
    for i,c in zip(results,coum_list):
        for j,k in zip(i[5],i[6]):
            rebuild_list.append([j,i[0],k,c,i[1],i[3],i[4]])
        pccs.append(i[3])
    pd.DataFrame(rebuild_list,columns=["target_neighbor","target","similarity","neighbors","alp","pcc","rmse"]).to_csv(f"./threshold/{threshold}_pcc_cpu_test_multi_target_threshold.csv")
    print(threshold, (sum(pccs) / len(pccs)))
    temp_list = []
    for i in results:
        for j,k,m in zip(i[7],i[8],cells_name):
            temp_list.append([i[0],m,j,k])
    pd.DataFrame(temp_list,columns=["drugs","cells_line","pridict","origin"]).sort_values("cells_line").to_csv(f"./threshold/{threshold}_pcc_cpu_test_multi_cells_line_threshold.csv")

run_parallel took 105.993887 seconds
0.5 0.43706421955539454
run_parallel took 101.271274 seconds
0.500956937799043 0.4451917959744461
run_parallel took 101.906251 seconds
0.5019138755980861 0.4451917959744461
run_parallel took 103.964190 seconds
0.5028708133971291 0.4453171114714625
run_parallel took 97.399715 seconds
0.5038277511961723 0.4532353521079988
run_parallel took 98.479172 seconds
0.5047846889952153 0.4531323530159655
run_parallel took 98.666881 seconds
0.5057416267942584 0.45318027017369533
run_parallel took 97.979485 seconds
0.5066985645933014 0.4568245265676322
run_parallel took 100.155832 seconds
0.5076555023923445 0.461036869868328
run_parallel took 100.302400 seconds
0.5086124401913875 0.4610023923722339
run_parallel took 97.347461 seconds
0.5095693779904307 0.46337888070872413
run_parallel took 95.106009 seconds
0.5105263157894737 0.46655040648425
run_parallel took 93.807194 seconds
0.5114832535885168 0.46780652130393996
run_parallel took 95.547330 seconds
0.512440191